## reading in data and importing things

In [1]:
#importing things
import pandas as pd
import os
import numpy as np
import csv
import pylab as pl
import matplotlib.pyplot as plt
import copy
import scipy
from scipy import stats
from scipy.stats import linregress
from scipy.stats import t
from scipy.stats import chi2
from scipy.stats import sem
from scipy.linalg import hadamard
from scipy.stats import ttest_ind_from_stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import ttest_ind_from_stats
from sklearn.manifold import TSNE
from collections import OrderedDict
import seaborn as sns
from functools import reduce
from scipy.optimize import curve_fit
from operator import itemgetter
import math

In [2]:
#reading in counts
rootdir = '../Raw_data/seq_counts/Binding/6261y'
list_of_dataframes = []
for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if '6261' in file:
                filepath = str(rootdir)+'/'+str(file)
                filename, file_extension = os.path.splitext(file)
                list_of_dataframes.append(pd.read_table(filepath,delimiter='\t\t',names=['variant',filename],index_col=None,dtype={'variant': str,'LogEC50':float}))

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['variant'],
                                            how='outer'), list_of_dataframes)
df_merged.fillna(0, inplace=True)
pd.DataFrame.to_csv(df_merged, '6261y_results/6261y_merged.txt', sep=',', na_rep=float(0), index=False)
df_merged

/Users/angelaphillips/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,variant,correct_counts_6261_y_9_4,correct_counts_6261_y_115_3,correct_counts_6261_y_115_2,correct_counts_6261_y_115_1,correct_counts_6261_y_9_2,correct_counts_6261_y_115_4,correct_counts_6261_y_9_3,correct_counts_6261_y_9_1,correct_counts_6261_y_8_4,...,correct_counts_6261_y_7_4,correct_counts_6261_y_10_3,correct_counts_6261_y_12_1,correct_counts_6261_y_10_2,correct_counts_6261_y_95_2,correct_counts_6261_y_12_2,correct_counts_6261_y_12_3,correct_counts_6261_y_10_1,correct_counts_6261_y_75_4,correct_counts_6261_y_95_1
0,11011111101,686.0,231.0,154.0,129.0,19.0,949.0,200.0,13.0,1640.0,...,1656.0,106.0,775.0,22.0,11.0,217.0,673.0,6.0,1444.0,7.0
1,11000111101,594.0,281.0,142.0,106.0,7.0,895.0,255.0,0.0,1214.0,...,1179.0,149.0,655.0,15.0,12.0,217.0,413.0,6.0,1063.0,1.0
2,11001111111,589.0,173.0,123.0,121.0,15.0,722.0,233.0,0.0,1211.0,...,1304.0,102.0,366.0,11.0,4.0,226.0,501.0,4.0,1069.0,11.0
3,11001111110,589.0,280.0,72.0,108.0,18.0,783.0,235.0,1.0,999.0,...,1245.0,135.0,495.0,22.0,13.0,366.0,366.0,4.0,1039.0,4.0
4,11001100111,589.0,219.0,160.0,148.0,36.0,787.0,170.0,1.0,1250.0,...,1211.0,107.0,543.0,34.0,15.0,239.0,239.0,10.0,1232.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,11010010000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,6.0,3.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
2044,10100110110,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,5.0,0.0,3.0,2.0,0.0,1.0,0.0,0.0,2.0,1.0
2045,00111110110,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2046,01001010001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,1.0,1.0,7.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0


In [3]:
#extracting mean log fluor and stdev log fluor from fcs files
rootdir = '../Raw_data/gate_events/6261y'

concs = ["_7_","_75_","_8_","_85_","_9_","_95_","_10_","_105_","_11_","_115_","_12_","_0_"]
PE1_bin_avg_list=[]
PE2_bin_avg_list=[]
PE3_bin_avg_list=[]
PE4_bin_avg_list=[]
PE1_bin_std_list=[]
PE2_bin_std_list=[]
PE3_bin_std_list=[]
PE4_bin_std_list=[]
PE_min = 0
PE_list=[]
for conc in concs:
    #calc min PE fluor to shift all above zero
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            filepath = subdir + os.sep + file
            if conc in filepath:
                with open(filepath, encoding="utf8", errors="ignore") as csv_file:
                    csv_reader = csv_file.readlines()
                    for line in csv_reader:
                        PE_list.append(float(line.split(",")[9]))

if min(PE_list) < PE_min:
    PE_min = min(PE_list)
else:
    PE_min = PE_min

for conc in concs:
    #bin mean log fluor
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + file
            if conc in filepath:
                if "PE1" in filepath:
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        PE_input = []
                        PE_adj = []
                        for lines in csv_reader:
                            PE_input.append(float(lines[9]))
                        for item in PE_input:
                            PE_a = item - PE_min + 1
                            PE_adj.append(float(PE_a))
                        PE_fluor = np.asarray(PE_adj)
                        PE_log_fluor = np.log10(PE_fluor)
                        bin_avg = np.average(PE_log_fluor)
                        bin_std = np.std(PE_log_fluor)
                        PE1_bin_avg_list.append(bin_avg)
                        PE1_bin_std_list.append(bin_std)
                        print(bin_avg)
                elif "PE2" in filepath:
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        PE_input = []
                        PE_adj = []
                        for lines in csv_reader:
                            PE_input.append(float(lines[9]))
                        for item in PE_input:
                            PE_a = item - PE_min + 1
                            PE_adj.append(float(PE_a))
                        PE_fluor = np.asarray(PE_adj)
                        PE_log_fluor = np.log10(PE_fluor)
                        bin_avg = np.average(PE_log_fluor)
                        bin_std = np.std(PE_log_fluor)
                        PE2_bin_avg_list.append(bin_avg)
                        PE2_bin_std_list.append(bin_std)
                        print(bin_avg)
                elif "PE3" in filepath:
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        PE_input = []
                        PE_adj = []
                        for lines in csv_reader:
                            PE_input.append(float(lines[9]))
                        for item in PE_input:
                            PE_a = item - PE_min + 1
                            PE_adj.append(float(PE_a))
                        PE_fluor = np.asarray(PE_adj)
                        PE_log_fluor = np.log10(PE_fluor)
                        bin_avg = np.average(PE_log_fluor)
                        bin_std = np.std(PE_log_fluor)
                        PE3_bin_avg_list.append(bin_avg)
                        PE3_bin_std_list.append(bin_std)
                        print(bin_avg)
                elif "PE4" in filepath:
                    with open(filepath, "r") as csv_file:
                        csv_reader = csv.reader(csv_file, delimiter=',')
                        PE_input = []
                        PE_adj = []
                        for lines in csv_reader:
                            PE_input.append(float(lines[9]))
                        for item in PE_input:
                            PE_a = item - PE_min + 1
                            PE_adj.append(float(PE_a))
                        PE_fluor = np.asarray(PE_adj)
                        PE_log_fluor = np.log10(PE_fluor)
                        bin_avg = np.average(PE_log_fluor)
                        bin_std = np.std(PE_log_fluor)
                        PE4_bin_avg_list.append(bin_avg)
                        PE4_bin_std_list.append(bin_std)
                        print(bin_avg)
    
with open('6261y_results/fluor_mean_std_6261y.csv', 'w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerows(zip(concs,PE1_bin_avg_list,PE2_bin_avg_list,PE3_bin_avg_list,PE4_bin_avg_list,PE1_bin_std_list,PE2_bin_std_list,PE3_bin_std_list,PE4_bin_std_list))

2.526258919131342
3.0369209438521505
3.744897374811607
4.3290931915238975
4.3284602516876
3.0485810928942585
3.755910850255042
2.519151446627974
4.322095758049585
3.7626402545578546
3.058514797247695
2.5048487587303403
4.298759605345208
3.0463466909892283
3.7349250116143082
2.493625605558625
2.495366834460165
3.7002600272611175
3.0400289668094547
4.234472464984739
4.0984042500717734
2.9828656930674025
3.549706307675503
2.4869938379017875
3.9516732401199017
2.4765389524547867
3.3923969284557884
2.9347213969348633
3.7086091396524283
3.1887513419012405
2.8626988735376866
2.4641968360811206
3.055315884466336
2.814369594939133
2.460026165481182
3.4494962791381814
3.159372982832906
2.9321961982723233
2.7889458892213024
2.441902825822653
2.433734481534633
2.8134519349797897
2.7576235271282536
2.9593897897013957
2.8240265183010855
2.7518832772055815
2.407406265879262
3.2177449192981173


In [3]:
#reading in fluor values and cell counts
fluor = pd.DataFrame(pd.read_csv('6261y_results/fluor_mean_std_6261y.csv',delimiter=',',names=['concen','1','2','3','4','1s','2s','3s','4s']))
cellct = pd.DataFrame(pd.read_csv('../Raw_data/cell_counts/Binding/6261y_cellct.csv',delimiter=','))

## calculate mean bin and read counts

In [4]:
concs = ['_7_','_75_','_8_','_85_','_9_','_95_','_10_','_105_','_11_','_115_','_12_','_0_']
i = 0 #index for conc
total_ct = []
bin1_cts = []
bin2_cts = []
bin3_cts = []
bin4_cts = []
for conc in concs:
    #first grab the counts and the fluor values
    #total cell counts in bin (a,b,c,d)
    Ca = float(cellct.at[i,'1'])
    Cb = float(cellct.at[i,'2'])
    Cc = float(cellct.at[i,'3'])
    Cd = float(cellct.at[i,'4'])
    bin1_ct = df_merged['correct_counts_6261_y'+str(conc)+'1'].sum(axis=0) #sum all counts in a bin
    avg_fluor_1 = fluor.at[i, '1'] #extract mean fluor
    bin2_ct = df_merged['correct_counts_6261_y'+str(conc)+'2'].sum(axis=0) #sum all counts in a bin
    avg_fluor_2 = fluor.at[i, '2'] #extract mean fluor
    bin3_ct = df_merged['correct_counts_6261_y'+str(conc)+'3'].sum(axis=0) #sum all counts in a bin
    avg_fluor_3 = fluor.at[i, '3'] #extract mean fluor
    bin4_ct = df_merged['correct_counts_6261_y'+str(conc)+'4'].sum(axis=0) #sum all counts in a bin
    avg_fluor_4 = fluor.at[i, '4'] #extract mean fluor
    total_ct_c = bin1_ct + bin2_ct + bin3_ct + bin4_ct #sum total count
    total_ct.append(total_ct_c) #append to list
    bin1_cts.append(bin1_ct)
    bin2_cts.append(bin2_ct)
    bin3_cts.append(bin3_ct)
    bin4_cts.append(bin4_ct)
    i = i + 1
    #now calc the var frequency
    df_merged['mean_bin1_%s'%conc] = (100*Ca*df_merged['correct_counts_6261_y'+str(conc)+'1']/bin1_ct)
    df_merged['mean_bin2_%s'%conc] = (100*Cb*df_merged['correct_counts_6261_y'+str(conc)+'2']/bin2_ct)
    df_merged['mean_bin3_%s'%conc] = (100*Cc*df_merged['correct_counts_6261_y'+str(conc)+'3']/bin3_ct)
    df_merged['mean_bin4_%s'%conc] = (100*Cd*df_merged['correct_counts_6261_y'+str(conc)+'4']/bin4_ct)
    #now calc the meanbin
    df_merged['meanbin_%s'%conc] = (df_merged['mean_bin1_%s'%conc] * float(avg_fluor_1) + df_merged['mean_bin2_%s'%conc] * float(avg_fluor_2) + df_merged['mean_bin3_%s'%conc] * float(avg_fluor_3) + df_merged['mean_bin4_%s'%conc] * float(avg_fluor_4))/(df_merged['mean_bin1_%s'%conc]+df_merged['mean_bin2_%s'%conc]+df_merged['mean_bin3_%s'%conc]+df_merged['mean_bin4_%s'%conc])
df_totals = pd.DataFrame(list(zip(concs, total_ct, bin1_cts, bin2_cts, bin3_cts, bin4_cts)), columns =['concen', 'total_ct', '1', '2','3','4'])
pd.DataFrame.to_csv(df_merged,'6261y_results/df_merged_6261y.csv')
pd.DataFrame.to_csv(df_totals,'6261y_results/df_totals_6261y.csv')
df_merged

,variant,correct_counts_6261_y_9_4,correct_counts_6261_y_115_3,correct_counts_6261_y_115_2,correct_counts_6261_y_115_1,correct_counts_6261_y_9_2,correct_counts_6261_y_115_4,correct_counts_6261_y_9_3,correct_counts_6261_y_9_1,correct_counts_6261_y_8_4,...,mean_bin1__12_,mean_bin2__12_,mean_bin3__12_,mean_bin4__12_,meanbin__12_,mean_bin1__0_,mean_bin2__0_,mean_bin3__0_,mean_bin4__0_,meanbin__0_
0,11011111101,686.0,231.0,154.0,129.0,19.0,949.0,200.0,13.0,1640.0,...,13613.313895,1060.414001,4160.008582,5867.400083,2.636450,61949.703473,0.000000,103.786307,4.077847,2.408156
1,11000111101,594.0,281.0,142.0,106.0,7.0,895.0,255.0,0.0,1214.0,...,11505.445937,1060.414001,2552.873023,4120.725696,2.614556,52392.147870,231.922489,286.307054,0.000000,2.411171
2,11001111111,589.0,173.0,123.0,121.0,15.0,722.0,233.0,0.0,1211.0,...,6428.997272,1104.394305,3096.826597,4420.155591,2.690014,45415.829911,3.865375,257.676349,68.507832,2.410996
3,11001111110,589.0,280.0,72.0,108.0,18.0,783.0,235.0,1.0,999.0,...,8694.955326,1788.532370,2262.352364,4120.725696,2.647437,47369.198939,3.865375,35.788382,11.417972,2.407944
4,11001100111,589.0,219.0,160.0,148.0,36.0,787.0,170.0,1.0,1250.0,...,9538.102509,1167.921411,1477.328456,4541.353406,2.632628,52671.200588,193.268740,21.473029,0.000000,2.408834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,11010010000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,52.696699,4.886700,0.000000,0.000000,2.461221,209.289539,0.000000,3.578838,0.000000,2.414411
2044,10100110110,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,52.696699,4.886700,0.000000,0.000000,2.461221,104.644769,0.000000,0.000000,0.000000,2.407406
2045,00111110110,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,52.696699,0.000000,0.000000,0.000000,2.433734,69.763180,0.000000,0.000000,0.000000,2.407406
2046,01001010001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,122.958964,0.000000,0.000000,0.000000,2.433734,279.052718,0.000000,0.000000,0.000000,2.407406


## calculate error in mean bin

In [5]:
#calculate error for each conc separately
concs = ['_7_','_75_','_8_','_85_','_9_','_95_','_10_','_105_','_11_','_115_','_12_','_0_']
i = 0 #index for conc
for conc in concs:
    #define concentration-specific variables
    #reads in bin (a,b,c,d)--for each seq
    df_merged['Ras_bin1_%s'%conc] = df_merged['correct_counts_6261_y'+str(conc)+'1']
    df_merged['Rbs_bin2_%s'%conc] = df_merged['correct_counts_6261_y'+str(conc)+'2']
    df_merged['Rcs_bin3_%s'%conc] = df_merged['correct_counts_6261_y'+str(conc)+'3']
    df_merged['Rds_bin4_%s'%conc] = df_merged['correct_counts_6261_y'+str(conc)+'4']
    #total reads in bin (a,b,c,d)
    Ra = float(df_totals.at[i,'1'])
    Rb = float(df_totals.at[i,'2'])
    Rc = float(df_totals.at[i,'3'])
    Rd = float(df_totals.at[i,'4'])
    #total cell counts in bin (a,b,c,d)
    Ca = float(cellct.at[i,'1'])
    Cb = float(cellct.at[i,'2'])
    Cc = float(cellct.at[i,'3'])
    Cd = float(cellct.at[i,'4'])
    #portion of cells in bin (a,b,c,d)--for each seq
    df_merged['Pas_bin1_%s'%conc] = ((df_merged['Ras_bin1_%s'%conc]*100*Ca)/Ra)/(((df_merged['Ras_bin1_%s'%conc]*100*Ca)/Ra)+((df_merged['Rbs_bin2_%s'%conc]*100*Cb)/Rb)+((df_merged['Rcs_bin3_%s'%conc]*100*Cc)/Rc)+((df_merged['Rds_bin4_%s'%conc]*100*Cd)/Rd))
    df_merged['Pbs_bin2_%s'%conc] = ((df_merged['Rbs_bin2_%s'%conc]*100*Cb)/Rb)/(((df_merged['Ras_bin1_%s'%conc]*100*Ca)/Ra)+((df_merged['Rbs_bin2_%s'%conc]*100*Cb)/Rb)+((df_merged['Rcs_bin3_%s'%conc]*100*Cc)/Rc)+((df_merged['Rds_bin4_%s'%conc]*100*Cd)/Rd))
    df_merged['Pcs_bin3_%s'%conc] = ((df_merged['Rcs_bin3_%s'%conc]*100*Cc)/Rc)/(((df_merged['Ras_bin1_%s'%conc]*100*Ca)/Ra)+((df_merged['Rbs_bin2_%s'%conc]*100*Cb)/Rb)+((df_merged['Rcs_bin3_%s'%conc]*100*Cc)/Rc)+((df_merged['Rds_bin4_%s'%conc]*100*Cd)/Rd))
    df_merged['Pds_bin4_%s'%conc] = ((df_merged['Rds_bin4_%s'%conc]*100*Cd)/Rd)/(((df_merged['Ras_bin1_%s'%conc]*100*Ca)/Ra)+((df_merged['Rbs_bin2_%s'%conc]*100*Cb)/Rb)+((df_merged['Rcs_bin3_%s'%conc]*100*Cc)/Rc)+((df_merged['Rds_bin4_%s'%conc]*100*Cd)/Rd))
    #mean log fluor for bin (a,b,c,d)--does not change w/seq
    Fa = fluor.at[i,'1']
    Fb = fluor.at[i,'2']
    Fc = fluor.at[i,'3']
    Fd = fluor.at[i,'4']
    #stdev log fluor for bin (a,b,c,d)--does not change w/seq
    Fae = fluor.at[i,'1s']
    Fbe = fluor.at[i,'2s']
    Fce = fluor.at[i,'3s']
    Fde = fluor.at[i,'4s']
    df_merged['termA'] = (((df_merged['Pas_bin1_%s'%conc]*df_merged['Pas_bin1_%s'%conc])/(df_merged['Ras_bin1_%s'%conc]))*(Fa*Fa))+((df_merged['Pas_bin1_%s'%conc]*df_merged['Pas_bin1_%s'%conc])*(Fae*Fae))
    df_merged['termB'] = (((df_merged['Pbs_bin2_%s'%conc]*df_merged['Pbs_bin2_%s'%conc])/(df_merged['Rbs_bin2_%s'%conc]))*(Fb*Fb))+((df_merged['Pbs_bin2_%s'%conc]*df_merged['Pbs_bin2_%s'%conc])*(Fbe*Fbe))
    df_merged['termC'] = (((df_merged['Pcs_bin3_%s'%conc]*df_merged['Pcs_bin3_%s'%conc])/(df_merged['Rcs_bin3_%s'%conc]))*(Fc*Fc))+((df_merged['Pcs_bin3_%s'%conc]*df_merged['Pcs_bin3_%s'%conc])*(Fce*Fce))
    df_merged['termD'] = (((df_merged['Pds_bin4_%s'%conc]*df_merged['Pds_bin4_%s'%conc])/(df_merged['Rds_bin4_%s'%conc]))*(Fd*Fd))+((df_merged['Pds_bin4_%s'%conc]*df_merged['Pds_bin4_%s'%conc])*(Fde*Fde))
    df_merged.fillna(0, inplace=True)
    df_merged['sum_%s'%conc] = df_merged['termA']+df_merged['termB']+df_merged['termC']+df_merged['termD']
    df_merged['error_%s'%conc] = np.sqrt(df_merged['sum_%s'%conc])
    i=i+1
df_merged

,variant,correct_counts_6261_y_9_4,correct_counts_6261_y_115_3,correct_counts_6261_y_115_2,correct_counts_6261_y_115_1,correct_counts_6261_y_9_2,correct_counts_6261_y_115_4,correct_counts_6261_y_9_3,correct_counts_6261_y_9_1,correct_counts_6261_y_8_4,...,Ras_bin1__0_,Rbs_bin2__0_,Rcs_bin3__0_,Rds_bin4__0_,Pas_bin1__0_,Pbs_bin2__0_,Pcs_bin3__0_,Pds_bin4__0_,sum__0_,error__0_
0,11011111101,686.0,231.0,154.0,129.0,19.0,949.0,200.0,13.0,1640.0,...,1776.0,0.0,29.0,5.0,0.998262,0.000000,0.001672,0.000066,0.011558,0.107507
1,11000111101,594.0,281.0,142.0,106.0,7.0,895.0,255.0,0.0,1214.0,...,1502.0,60.0,80.0,0.0,0.990206,0.004383,0.005411,0.000000,0.011960,0.109364
2,11001111111,589.0,173.0,123.0,121.0,15.0,722.0,233.0,0.0,1211.0,...,1302.0,1.0,72.0,84.0,0.992785,0.000084,0.005633,0.001498,0.012606,0.112275
3,11001111110,589.0,280.0,72.0,108.0,18.0,783.0,235.0,1.0,999.0,...,1358.0,1.0,10.0,14.0,0.998923,0.000082,0.000755,0.000241,0.012575,0.112139
4,11001100111,589.0,219.0,160.0,148.0,36.0,787.0,170.0,1.0,1250.0,...,1510.0,50.0,6.0,0.0,0.995940,0.003654,0.000406,0.000000,0.012076,0.109890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,11010010000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,6.0,0.0,1.0,0.0,0.983188,0.000000,0.016812,0.000000,0.944039,0.971617
2044,10100110110,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,3.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.000000,1.940202,1.392911
2045,00111110110,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.000000,2.906137,1.704739
2046,01001010001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,8.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.000000,0.732785,0.856028


## fitting Kd

## fitting Kd for entire dataframe

In [9]:
meanbins_errors = df_merged[['variant', 'meanbin__7_', 'meanbin__75_', 'meanbin__8_', 'meanbin__85_', 'meanbin__9_', 'meanbin__95_', 'meanbin__10_', 'meanbin__105_', 'meanbin__11_', 'meanbin__115_','meanbin__12_','meanbin__0_','error__7_','error__75_','error__8_','error__85_','error__9_','error__95_','error__10_','error__105_','error__11_','error__115_','error__12_','error__0_']].copy()
pd.DataFrame.to_csv(meanbins_errors, '6261y_results/6261y_meanbins_errors.csv', sep=',', index=False, header=None)
meanbins_errors

,variant,meanbin__7_,meanbin__75_,meanbin__8_,meanbin__85_,meanbin__9_,meanbin__95_,meanbin__10_,meanbin__105_,meanbin__11_,...,error__8_,error__85_,error__9_,error__95_,error__10_,error__105_,error__11_,error__115_,error__12_,error__0_
0,11011111101,4.261384,4.255587,4.252155,4.192205,4.154954,4.027642,3.895399,3.626868,3.335197,...,0.192520,0.189067,0.213729,0.200251,0.237825,0.194636,0.153782,0.119421,0.100973,0.107507
1,11000111101,4.224830,4.239427,4.232244,4.198411,4.148430,4.026118,3.870888,3.634974,3.335604,...,0.193205,0.194693,0.214444,0.202807,0.236774,0.200209,0.155251,0.118373,0.108116,0.109364
2,11001111111,4.242338,4.247089,4.239267,4.208006,4.149326,4.024144,3.899060,3.620682,3.351454,...,0.195991,0.198751,0.216198,0.206283,0.243022,0.201524,0.162879,0.129816,0.105500,0.112275
3,11001111110,4.222112,4.223673,4.224771,4.173016,4.146146,4.017790,3.862555,3.622058,3.319807,...,0.197308,0.189971,0.215552,0.203173,0.241248,0.199265,0.161703,0.124269,0.105400,0.112139
4,11001100111,4.231727,4.253058,4.243731,4.203645,4.153946,4.013535,3.875657,3.596595,3.273280,...,0.197719,0.195044,0.220992,0.202730,0.237785,0.191601,0.148981,0.123492,0.113258,0.109890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,11010010000,3.036921,3.327890,3.631979,3.046347,3.116069,3.142124,3.392397,2.751361,2.570610,...,1.640840,3.052952,2.260195,1.676930,1.391727,0.591216,0.879877,0.000000,1.311991,0.971617
2044,10100110110,4.032215,4.177037,3.762640,4.298760,3.700260,2.999530,2.934721,2.862699,2.876192,...,3.766861,2.157692,3.704097,1.769984,2.078873,2.025821,2.036399,0.000000,1.311991,1.392911
2045,00111110110,0.000000,4.328460,2.504849,3.545929,3.700260,2.982866,0.000000,0.000000,2.460026,...,2.508182,2.153425,3.704097,2.986991,0.000000,0.000000,2.463589,0.000000,1.410664,1.704739
2046,01001010001,3.565508,3.734985,3.457464,3.585140,2.495367,2.993170,3.300496,2.589311,2.819094,...,1.116100,1.818523,1.768790,1.580307,1.829846,0.869772,1.629984,0.000000,0.928315,0.856028


In [6]:
#plotting with error
# define function
def extractKd(concentrations, bins_mean):
    """
        Arguments: concentrations and exponential mean bin numbers (bin number: 1, 2, 3, 4)
        Return: -log10(Kd), and the r^2
    """

    def fun(c, Kd, A, B):
        return np.log10(A * (10**c/((10**c)+(10**Kd))) + B)
    popt, pcov = scipy.optimize.curve_fit(fun,concentrations,
                                          bins_mean,
                                          p0=[(-9),10**(4),10**(1)],
                                          sigma=error_new, absolute_sigma=True,
                                          bounds = [((-14),10**3,1),((-5),10**(5),10**(3))],
                                          maxfev=400000)
    return(-1*popt[0],popt[1],popt[2], 1 - np.sum((fun(concentrations, *popt) - bins_mean)**2)/np.sum((bins_mean - bins_mean.mean())**2),np.sqrt(np.diag(pcov))[0])




lines = []
with open('6261y_results/6261y_meanbins_errors.csv', 'r') as f:
    for line in f.readlines():
        variant, meanbin__7_, meanbin__75_, meanbin__8_, meanbin__85_, meanbin__9_, meanbin__95_, meanbin__10_, meanbin__105_, meanbin__11_, meanbin__115_,meanbin__12_,meanbin__0_,error__7_,error__75_,error__8_,error__85_,error__9_,error__95_,error__10_,error__105_,error__11_,error__115_,error__12_,error__0_ = line.strip().split(',')
        lines.append((variant, meanbin__7_, meanbin__75_, meanbin__8_, meanbin__85_, meanbin__9_, meanbin__95_, meanbin__10_, meanbin__105_, meanbin__11_, meanbin__115_,meanbin__12_,meanbin__0_,error__7_,error__75_,error__8_,error__85_,error__9_,error__95_,error__10_,error__105_,error__11_,error__115_,error__12_,error__0_))

from operator import itemgetter
import numpy as np
import math
import csv
concentrations = np.array((-7.02,-7.52,-8.02,-8.52,-9.02,-9.52,-10.02,-10.52,-11.02,-11.52,-12.02,-14))
for line in lines:
    print(line)
    output = []
    geno, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, e1, e2, e3, e4, e5, e6, e7, e8, e9, e10, e11, e12 = itemgetter(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24)(line)
    bins_mean = np.array((float(c1), float(c2), float(c3), float(c4), float(c5), float(c6), float(c7), float(c8), float(c9), float(c10), float(c11), float(c12)))
    error = np.array((float(e1), float(e2), float(e3), float(e4), float(e5), float(e6), float(e7), float(e8), float(e9), float(e10), float(e11), float(e12)))
    #remove concs with no counts
    index = np.argwhere(bins_mean == 0)
    bins_mean_new = np.delete(bins_mean, index)
    conc_new = np.delete(concentrations, index)
    error_new = np.delete(error, index)
    res = extractKd(conc_new, bins_mean_new)
    output.append(geno)
    output.append(str(res[0]))
    output.append(str(res[1]))
    output.append(str(res[2]))
    output.append(str(res[3]))
    output.append(str(res[4]))
    output.append(c1)
    output.append(c2)
    output.append(c3)
    output.append(c4)
    output.append(c5)
    output.append(c6)
    output.append(c7)
    output.append(c8)
    output.append(c9)
    output.append(c10)
    output.append(c11)
    output.append(c12)
    output.append(e1)
    output.append(e2)
    output.append(e3)
    output.append(e4)
    output.append(e5)
    output.append(e6)
    output.append(e7)
    output.append(e8)
    output.append(e9)
    output.append(e10)
    output.append(e11)
    output.append(e12)
    print(output)
    import csv
    csvOut = open('6261y_results/6261y_kd_meanbin_err_keepall.csv', 'a+')
    wr = csv.writer(csvOut, delimiter=',')
    wr.writerow(output)
    csvOut.close()

('11011111101', '4.261383905042267', '4.25558748718471', '4.252154831821839', '4.1922048131309175', '4.154953501387623', '4.027641730197654', '3.895398860720674', '3.626867556337622', '3.335197425161665', '2.8684958175026725', '2.636450430669944', '2.408156277785568', '0.19556712180377128', '0.19688291031075997', '0.19252006120445872', '0.18906729638384462', '0.2137292792566615', '0.20025077185119602', '0.23782528723049676', '0.1946359885493834', '0.1537823796945376', '0.11942071479993337', '0.10097268564114414', '0.10750741167301048')
['11011111101', '10.049260242324802', '16389.356285502956', '254.1527896727248', '0.9960396048397343', '0.15771487912786422', '4.261383905042267', '4.25558748718471', '4.252154831821839', '4.1922048131309175', '4.154953501387623', '4.027641730197654', '3.895398860720674', '3.626867556337622', '3.335197425161665', '2.8684958175026725', '2.636450430669944', '2.408156277785568', '0.19556712180377128', '0.19688291031075997', '0.19252006120445872', '0.1890672

['01000100111', '9.710012069214416', '12017.619121690772', '240.73378685299556', '0.9961063593389803', '0.1453510521347534', '4.124666304526029', '4.095246827215659', '4.10504837749248', '4.0822312664323315', '3.9617515140741943', '3.796752314926831', '3.6113275252989068', '3.290352287496653', '2.9379566385327216', '2.5526838076375356', '2.4395873256426395', '2.4079633083302356', '0.1794101463299051', '0.16653734188765568', '0.16841833033555076', '0.17097659635797247', '0.17786790560746504', '0.15908578024136566', '0.1729649481123805', '0.12583203097850149', '0.09853674799675971', '0.15072284540155992', '0.13788666761307652', '0.10530718547644176']
('10001100111', '4.073660238688743', '4.077274396835519', '4.054037798295404', '4.005028094311779', '3.9281507363087353', '3.6852140460353335', '3.453356500366881', '3.0510518705923335', '2.6720886879934884', '2.4860479392320776', '2.4429594206266607', '2.408228697561563', '0.17267702025938111', '0.16618275740907928', '0.16505145297532553', 

['01001111101', '9.81375541977757', '12238.270890221987', '237.91413872078104', '0.996547290155896', '0.1519939212014212', '4.12089503802499', '4.141410924534155', '4.084732434884643', '4.069324551084686', '4.008380653094887', '3.861667849093373', '3.6734740006400926', '3.3606663164712294', '3.0278180105205634', '2.610703919684436', '2.441556046644689', '2.4078046296869484', '0.18732061985225532', '0.1747154568662609', '0.16813214475271238', '0.17545158694811644', '0.1958183180675861', '0.16668451927035435', '0.19615465828508433', '0.13777499165039778', '0.11133330928169145', '0.15527134594968703', '0.13930892437261064', '0.10827980256392822']
('10011111100', '4.146751291976092', '4.145772667709652', '4.110875880862585', '4.044642514917489', '3.9913388094627984', '3.857369409787358', '3.6606389590698516', '3.3499783809429755', '3.012964666563035', '2.630545081839868', '2.470417856165584', '2.411653615619603', '0.17876899406415772', '0.17811410180463288', '0.17285378461397116', '0.17044

['10011101101', '9.232448264118304', '8891.64738456587', '256.6146453442106', '0.9977001289234708', '0.15271457948290762', '3.9521268795591187', '4.008873021893965', '3.937311485604036', '3.841609092868811', '3.7807421423971888', '3.4617454422555873', '3.2254220850000572', '2.8347184040031164', '2.6056126359271454', '2.4711698082076614', '2.435531153624034', '2.4075824802802708', '0.16153256443774114', '0.15960931841712342', '0.15066192355527444', '0.14793710222999196', '0.16679858192424965', '0.13249617338487651', '0.12451151412667288', '0.09449999413684698', '0.11706077189455456', '0.1910193044181372', '0.14130637488336498', '0.10735585764393857']
('10011111111', '4.170145694047928', '4.155873506245181', '4.139388710327538', '4.0999862171940835', '4.063241690579495', '3.8932084279675316', '3.765123288522247', '3.484425301413431', '3.1745781532512627', '2.756509357903449', '2.531048844846192', '2.4077617692380593', '0.20264931472883188', '0.1875697596367705', '0.17706257279001372', '0

['11011100010', '9.588501564689375', '11899.223727160888', '251.60983545522382', '0.9990797615079813', '0.16431590200602741', '4.097234574606489', '4.09481958691137', '4.053788922810029', '4.058634945363743', '3.9806836667141403', '3.793613990853841', '3.56997563732578', '3.1511190482097993', '2.8576601661270176', '2.589964403852927', '2.443212475353997', '2.4087243551319566', '0.19749054420066264', '0.18289211737940964', '0.1846402499923389', '0.1900890190876792', '0.20895115279709683', '0.18088721956592815', '0.19449968505263915', '0.1239841239280463', '0.11472765175127464', '0.16497687303220102', '0.14750822503567168', '0.11513902236265607']
('11011011011', '4.006887952614728', '4.004892170704244', '3.9857230631839644', '3.8589009735910125', '3.896443101857037', '3.7017607314358165', '3.562959181426012', '3.2979050120258253', '2.942040456158066', '2.5518695946904866', '2.4433918821960634', '2.407535354624383', '0.17838558959821021', '0.16550284336457385', '0.16502415081172553', '0.1

...

['11111110100', '9.71441244766973', '7771.626539437981', '256.25458573294975', '0.8994835786725314', '1.9896333196428373', '3.8716871344920234', '3.755910850255041', '4.136732352280625', '4.031145731992126', '3.400447516858849', '3.975823845627857', '3.3582442501302188', '2.952076929815871', '3.002433219712677', '2.6292898109996825', '2.433734481534633', '2.4074062658792617', '2.2813940664043657', '2.1761430278999905', '2.397721555383039', '1.2905400932905453', '1.7347305548622114', '2.0074327270628767', '1.6805651066483054', '1.710957895323738', '1.52555838394194', '1.6947215698006832', '0.9283154690284626', '1.2071600061678156']
('11001010000', '0.0', '3.9907286485274036', '3.7224452044688916', '3.9560478640799217', '0.0', '2.629819290680495', '3.0950268555334812', '2.711573718754936', '2.8143695949391336', '2.9321961982723237', '2.4337344815346333', '2.4074062658792617', '0.0', '2.3713136359310774', '2.698737875220184', '2.3646619106942315', '0.0', '1.335568611369796', '1.7996234505

['00111010011', '6.955696762649609', '1000.0000000095188', '344.744573864157', '0.506265448118986', '3.981150040907797', '2.8329799045019106', '2.778006718073375', '2.613406452910531', '2.707363045814222', '2.759673686913198', '2.543643482806478', '2.618909859098327', '2.479039915378824', '2.4600261654811817', '2.441902825822653', '2.4337344815346333', '2.4074062658792617', '0.3989708636861815', '0.3789160062712226', '0.34026198169644317', '0.3904439592292001', '0.36873881085699206', '0.41979033553611', '0.6102972841777762', '0.5011156650918679', '0.5003004520888504', '0.7186530222495915', '0.41411081906915687', '0.3461040149790294']
('00010010000', '2.644504525224291', '2.7131496450069044', '2.5973731328951106', '2.607332152706901', '2.5527858105922947', '2.517468277594538', '2.4765389524547867', '2.4845178734318507', '2.4600261654811817', '2.441902825822653', '2.4337344815346333', '2.408989494783714', '0.3605199835702381', '0.33481990008428575', '0.4279507632919088', '0.3338084873367

['00001110111', '9.671254942973393', '5233.8209423239405', '230.4340540388678', '0.8041724283087504', '2.268013067687387', '3.953256164825496', '3.9372462063419555', '3.307960268780777', '4.298759605345208', '3.400447516858849', '3.2315201906742796', '3.666931594159395', '3.031178311449461', '2.460026165481182', '2.441902825822653', '2.4337344815346333', '2.4074062658792617', '2.028106373796977', '2.023314937136215', '1.9362533497642989', '3.0455653373470852', '2.449955948237214', '2.007083392139549', '2.295707072130011', '1.52990454233378', '1.7445360736574127', '1.10096893582148', '1.4106637397107793', '1.2071600061678156']
('00001010010', '2.8457536489904762', '2.689291056583376', '2.901649416965957', '2.6798377408452465', '2.6388225604875153', '2.510053940065426', '2.5458014648341303', '2.4897290966883387', '2.4665999160151313', '2.441902825822653', '2.4337344815346333', '2.4086332648802906', '0.437881473311743', '0.3832486262199352', '0.5202950801891955', '0.46560079362088586', '0

('00110010010', '2.6068182506744724', '2.860644526298269', '2.84128541588582', '2.5587049244510096', '2.7509047753634466', '2.4869938379017875', '2.4765389524547867', '2.4641968360811206', '2.460026165481182', '2.441902825822653', '2.4337344815346333', '2.4074062658792617', '0.9980958057553464', '0.8330528496970394', '1.2852710637457154', '0.9746285925876774', '0.9797267132673607', '1.118754379115925', '1.0184439146600017', '1.1091483629754575', '2.4635889429281916', '1.7323392914992373', '1.223267574701973', '1.0804883312842133')
['00110010010', '6.837138889001856', '1000.0000000000001', '340.4740539077399', '0.17968936787644985', '13.091133734722035', '2.6068182506744724', '2.860644526298269', '2.84128541588582', '2.5587049244510096', '2.7509047753634466', '2.4869938379017875', '2.4765389524547867', '2.4641968360811206', '2.460026165481182', '2.441902825822653', '2.4337344815346333', '2.4074062658792617', '0.9980958057553464', '0.8330528496970394', '1.2852710637457154', '0.9746285925

['11010010000', '9.977584033381284', '1697.2941732789925', '250.56183669402674', '0.7479162065105672', '1.6727161768662175', '3.0369209438521505', '3.327890072706098', '3.6319790695780667', '3.0463466909892283', '3.116068976232857', '3.142123558637466', '3.3923969284557884', '2.7513610158516832', '2.5706104941105226', '0.0', '2.461220680088726', '2.414410671536015', '1.2552014484832499', '1.116407039046423', '1.6408403580047517', '3.0529518157800233', '2.2601951479159723', '1.67693047621392', '1.3917268460210455', '0.5912155285043399', '0.8798767843170751', '0.0', '1.3119909313872902', '0.9716165808310655']
('10100110110', '4.03221502389329', '4.177036912755125', '3.762640254557855', '4.298759605345208', '3.7002600272611175', '2.9995298425612837', '2.934721396934864', '2.8626988735376866', '2.8761923706957724', '0.0', '2.461220680088726', '2.4074062658792617', '1.5702577525417867', '2.4649524892610226', '3.766860732995515', '2.1576919817047897', '3.704096956830622', '1.7699840471344077